In [ ]:
from libcpab.pytorch import cpab
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('class_1.txt')
file_list = df['file_name']

In [ ]:
df.head(5)

In [ ]:
for i in file_list[0:5]:
        img = plt.imread('class_1/'+ i) / 255
        #print(np.shape(img))
        data = np.tile(np.expand_dims(img, 0), [1,1,1,1])
        #print(np.shape(data))# create batch of data

        # Convert to torch tensor and torch format [n_batch, n_channels, width, height]
        data = torch.Tensor(data).permute(0,3,1,2)
        #print(np.shape(data)) # data 
        data = np.reshape(data,(1,1,28,28))
        #print(np.shape(data)) # data 
        
        img2 = plt.imread('class_1/'+ random.choice(file_list)) / 255
        data2 = np.tile(np.expand_dims(img2, 0), [1,1,1,1])
        #print(np.shape(data2))# create batch of data

        # Convert to torch tensor and torch format [n_batch, n_channels, width, height]
        data2 = torch.Tensor(data2).permute(0,3,1,2)
        #print(np.shape(data2)) # data 
        data2 = np.reshape(data2,(1,1,28,28))
        #print(np.shape(data2)) # data 
        

        
        T2 = cpab(tess_size=[3,3], device='cpu')
        theta_est = T2.identity(1, epsilon=1e-4)
        theta_est.requires_grad = True
        #print(np.shape(theta_est))

        # Pytorch optimizer
        optimizer = torch.optim.Adam([theta_est], lr=0.1)

        # Optimization loop
        maxiter = 100
        for i in range(maxiter):
            trans_est = T2.transform_data(data, theta_est, outsize=(28, 28))
            loss = (data2.to(trans_est.device) - trans_est).pow(2).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

        # Show the results
        plt.subplots(1,3, figsize=(10, 15))
        plt.subplot(1,3,1)
        #print(np.shape(data))
        #print('the transformed shape: {}'.format(np.shape(trans_est)))
        data = np.reshape(data,(1,28,28))
        plt.imshow(data.permute(0,1,2).numpy()[0])
        plt.axis('off')
        plt.title('Source')
        plt.subplot(1,3,2)
        data2 = np.reshape(data2,(1,28,28))
        plt.imshow(data2.permute(0,1,2).cpu().numpy()[0])
        plt.axis('off')
        plt.title('Target')
        plt.subplot(1,3,3)
        trans_est = torch.reshape(trans_est,(1,28,28))
        plt.imshow(trans_est.permute(0,1,2).cpu().detach().numpy()[0])
        plt.axis('off')
        plt.title('Estimate')
        plt.show()

In [ ]:
image_list = file_list[0:10]
N = len(image_list)
thetas = np.zeros((N*(N-1),34))
counter = 0
for m in range(len(image_list)):
    for n in range(len(image_list)):
        if m != n:
            data = np.tile(np.expand_dims(plt.imread('class_1/'+ image_list[m]) / 255, 0), [1,1,1,1]) # read an image
            # Convert to torch tensor and torch format [n_batch, n_channels, width, height]
            data = torch.Tensor(data).permute(0,3,1,2)
            data = np.reshape(data,(1,1,28,28))
            
            # read 2nd image
            data2 = np.tile(np.expand_dims(plt.imread('class_1/'+ image_list[n]) / 255, 0), [1,1,1,1]) 
            data2 = torch.Tensor(data2).permute(0,3,1,2)
            data2 = np.reshape(data2,(1,1,28,28))
            
            # Now, create pytorch procedure that enables us to estimate the transformation
            # we have just used for transforming the data
            T2 = cpab(tess_size=[3,3], device='cpu')
            theta_est = T2.identity(1, epsilon=1e-4)
            theta_est.requires_grad = True

            # Pytorch optimizer
            optimizer = torch.optim.Adam([theta_est], lr=0.1)
            # Optimization loop
            maxiter = 100
            for i in range(maxiter):
                trans_est = T2.transform_data(data, theta_est, outsize=(28, 28))
                loss = (data2.to(trans_est.device) - trans_est).pow(2).mean()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #print('Iter', i, ', Loss', np.round(loss.item(), 4), ', ||theta_true - theta_est||: ',
                      #np.linalg.norm((data2-trans_est.cpu().detach()).numpy().round(4)))
            thetas[counter,:] = torch.Tensor.cpu(theta_est).detach().numpy()[:]
            counter += 1 

In [ ]:
mean_thetas = thetas.mean(0)
cov_thetas = np.cov(thetas, rowvar=False)
multi_gausian = np.random.multivariate_normal(mean_thetas, cov_thetas, 34).T
multi_gausian = torch.tensor(multi_gausian.astype(np.float32))
np.shape(multi_gausian)

In [ ]:
true_theta = multi_gausian[:1]

In [ ]:
np.shape(true_theta)


In [ ]:
# generating new images

for i in file_list[0:10]:
        data = plt.imread('class_1/'+ i) / 255
        #print(np.shape(img))
        data = np.tile(np.expand_dims(data, 0), [1,1,1,1])
        # Convert to torch tensor and torch format [n_batch, n_channels, width, height]
        data = torch.Tensor(data).permute(0,3,1,2)
        data = np.reshape(data,(1,1,28,28))
        
        T2 = cpab(tess_size=[3,3], device='cpu')
        
        trans_est = T2.transform_data(data,true_theta , outsize=(28, 28))
        
        # Show the results
        plt.subplots(1,2, figsize=(10, 15))
        plt.subplot(1,2,1)
        
        data = np.reshape(data,(1,28,28))
        plt.imshow(data.permute(0,1,2).numpy()[0])
        plt.axis('off')
        plt.title('Source')
        
        plt.subplot(1,2,2)
        trans_est = torch.reshape(trans_est,(1,28,28))
        plt.imshow(trans_est.permute(0,1,2).cpu().detach().numpy()[0])
        plt.axis('off')
        plt.title('New data')
        plt.show()